# **OVERVIEW OF THE DATASET**

- Definition of sleep event: 
    - Event class: Onset/Wakeup
    - Sleep Onset is indicate the beginning of the sleep, and wakeup is defined as the ebd of the sleep.
    - Single sleep period must be the longest inactive period while the accelerometer is worn. 
    - Single sleep must be the at leat 30 minutes in length which can be interrupted by bouts that do not exceed 30 consecutive 30 mm,inutes.
    
    
- The individual can have multiple sleep window in same clander day. But the **longest sleep window during the night will be recored**.


- There is no strict rule to limit number of sleep events in a givien period **however only one window assign per nigth**.


- The event that annotated when the accelerometer removed should be refrain from makking predictions. 

- **Target variable: Event (Oneset/Wakeup)**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import polars as pl

- Since the dataset is quite big, Used polars to load fast and effectively in memory.
- Converted the outpiut into pandas dataframe to perform EDA and analysis.

In [2]:
# reading train_series parquet file as well as converting the timestamp to datetime dtype
train_series = (pl.scan_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet")
    .with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z"))
    .collect().to_pandas())

# reading train_event csv
train_event = (pl.scan_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
              .with_columns(pl.col('timestamp').str.strptime(pl.Datetime,"%Y-%m-%dT%H:%M:%S%Z"))
              .collect().to_pandas())

In [3]:
# understanding the features 
train_series.info()
print(f"\n The shape of the Dataset is : \n rows = {train_series.shape[0]}, columns = {train_series.shape[1]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127946340 entries, 0 to 127946339
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   series_id  object        
 1   step       uint32        
 2   timestamp  datetime64[ns]
 3   anglez     float32       
 4   enmo       float32       
dtypes: datetime64[ns](1), float32(2), object(1), uint32(1)
memory usage: 3.3+ GB

 The shape of the Dataset is : 
 rows = 127946340, columns = 5


In [4]:
# understanding the features 
train_event.info()
print(f"\n The shape of the Dataset is : \n rows = {train_event.shape[0]}, columns = {train_event.shape[1]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14508 entries, 0 to 14507
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   series_id  14508 non-null  object        
 1   night      14508 non-null  int64         
 2   event      14508 non-null  object        
 3   step       9585 non-null   float64       
 4   timestamp  9585 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 566.8+ KB

 The shape of the Dataset is : 
 rows = 14508, columns = 5


# Getting insight from Train_series

In [5]:
# display the top 5 rows
train_series.head()

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14 15:30:00,2.6367,0.0217
1,038441c925bb,1,2018-08-14 15:30:05,2.6368,0.0215
2,038441c925bb,2,2018-08-14 15:30:10,2.6370,0.0216
3,038441c925bb,3,2018-08-14 15:30:15,2.6368,0.0213
4,038441c925bb,4,2018-08-14 15:30:20,2.6368,0.0215


In [6]:
# Total unique series_id
train_series['series_id'].nunique()

277

In [7]:
# Five point summary
train_series.describe()

,step,timestamp,anglez,enmo
count,1.279463e+08,127946340,1.279463e+08,1.279463e+08
mean,2.548048e+05,2018-07-16 13:31:51.766071296,-8.810453e+00,4.131503e-02
min,0.000000e+00,2017-08-04 17:30:00,-9.000000e+01,0.000000e+00
25%,1.158120e+05,2018-02-12 20:09:30,-3.185890e+01,1.300000e-03
50%,2.345190e+05,2018-07-22 02:40:47.500000,-9.597900e+00,1.720000e-02
75%,3.571960e+05,2018-12-24 18:28:05,1.130020e+01,4.370000e-02
max,1.433879e+06,2019-07-09 09:44:55,9.000000e+01,1.143370e+01
std,1.778930e+05,NaN,3.552188e+01,1.018289e-01
